In [2]:
import numpy as np
from tensorflow.keras.datasets import fashion_mnist
from tqdm import tqdm
import decimal
import tensorflow as tf
import wandb as wb

In [3]:

def sigmoid(z):
    z = np.clip(z,-500,500)
    return 1.0 / (1 + np.exp(-(z)))


def tanh(z):
    return np.tanh(z)

# def relu(z):
#     return (z>0)*(z) + ((z<0)*(z)*0.01)
#     #return np.maximum(z,0)
#     #return np.where(z<0, 0.01*z, z)
def relu(z):
    return np.maximum(z*0.01,z)


def softmax(Z):
    ep = 1e-5
    Z = np.clip(Z,-600,600)
    return (np.exp(Z) / (np.sum(np.exp(Z)))+ep)

# def softmax(Z):
#     c = Z.max()
#     logsumexp = np.log(np.exp(Z - c).sum())
#     return Z - c - logsumexp


def grad_sigmoid(z):
    return  sigmoid(z)*(1-sigmoid(z))

def grad_tanh(z):
    return 1 - np.tanh(z) ** 2


# def grad_relu(z):
#     return (z>0)*np.ones(z.shape) + (z<0)*(0.01*np.ones(z.shape) )

def grad_relu(x):
    alpha=0.01
    dx = np.ones_like(x)
    dx[x < 0] = alpha
    return dx

## sizeHL is a number it can be converted back to list
class FeedForwardNN:
    def __init__(
        self, layers, sizeHL, x_train, y_train, x_val, y_val, 
        x_test, y_test, batchSize, lr, iterations, activation,
        initializer, loss, weight_decay):
        self.classes = 10
        self.layers = layers+2
        self.layersSize = [784] + [sizeHL for i in range(layers)] + [10]
        self.total = x_train.shape[0]
        # 784 * 60000
        self.X = np.transpose(x_train.reshape(x_train.shape[0],x_train.shape[1]*x_train.shape[2]))
        self.X_test = np.transpose(x_test.reshape(x_test.shape[0],x_test.shape[1]*x_test.shape[2]))
        self.X_val = np.transpose(x_val.reshape(x_val.shape[0],x_val.shape[1]*x_val.shape[2]))
        #normalizing values
        self.X = self.X/255
        self.X_test = self.X_test/255
        self.X_val = self.X_val/255
        # changing normal values to one hot encoding
        # shape will be 10*60000
        
        self.Y = self.oneHotEncoder(y_train)
        self.Y_test = self.oneHotEncoder(y_test)
        self.Y_val = self.oneHotEncoder(y_val)
        self.y_train = y_train
        self.y_val = y_val
        self.y_test = y_test
        self.Activations_dict = {"sigmoid": sigmoid, "tanh": tanh, "relu": relu}
        self.derActivation_dict = {
            "sigmoid": grad_sigmoid,
            "tanh": grad_tanh,
            "relu": grad_relu,
        }

        self.Initializer_dict = {
            "xavier": self.xavier,
            "random": self.random,
        }

        # self.Optimizer_dict = {
        #     "SGD": self.sgd,
        #     "MGD": self.mgd,
        #     "NAG": self.nag,
        #     "RMSPROP": self.rmsProp,
        #     "ADAM": self.adam,
        #     "NADAM": self.nadam,
        # }
        self.activation = self.Activations_dict[activation]
        self.derivation_activation = self.derActivation_dict[activation]
    #    self.optimzer = self.Optimizer_dict[optimizer]
        self.initializer = self.Initializer_dict[initializer]
        self.lossFunction = loss
        self.weightDecay = weight_decay
        self.epochs = iterations
        self.batchSize = batchSize
        self.lr = lr

        # initializing weights and biases
        self.weights, self.biases = self.initializeWeights(self.layersSize)


    def train(self,optimizer,momentum=0.9,beta1=0.9,beta2=0.999):
        if(optimizer=='sgd'):
            self.sgd()
        elif(optimizer=='mgd'):
            self.mgd(momentum)
        elif(optimizer=='nag'):
            self.nag(momentum)
        elif(optimizer=='rmsprop'):
            self.rmsProp(momentum)
        elif(optimizer=='adam'):
            self.adam(beta1,beta2)
        elif(optimizer=='nadam'):
            self.nadam(beta1,beta2)


    def oneHotEncoder(self,rawY):
        onehot = np.zeros((self.classes,rawY.shape[0]))
        size = rawY.shape[0]
        for i in range(size):
            onehot[int(rawY[i])][i] = 1.0
        return onehot
    


    def accuracyLoss(self,x,y):
        pred,H,A = self.forwardPropagation(x)
        pred = pred.T
        count =0
        crossloss = []
        for i in range(len(y)):
            if(np.argmax(pred[i])==y[i]):
                count+=1
        loss  = -np.mean(np.sum(self.oneHotEncoder(y).T * np.log(pred + 1e-15), axis=1))
        return count/(len(y)),loss

    # def meanSquaredErrorLoss( y, pred):
    #     mse = np.mean((y - pred) ** 2)
    #     return mse

    def crossEntropyLoss( y, pred):
        CE = [-Y_true[i] * np.log(Y_pred[i]) for i in range(len(Y_pred))]
        crossEntropy = np.mean(CE)
        return crossEntropy

    def initializeWeights(self,layersSize):
        weights =[]
        biases =[]
        for i in range(len(layersSize)-1):
            weights.append(self.initializer([layersSize[i+1],layersSize[i]]))
            biases.append(self.initializer([layersSize[i+1],1]))
        return weights,biases

    def xavier(self, size):
        std = np.sqrt(2 / (size[0] + size[1]))
        # size[0] = next layer's neurons
        # size[1] = prev layer's neurons
        return np.random.normal(0, std, size=(size[0], size[1]))

    def random(self, size):
        return np.random.normal(0, 1, size=(size[0], size[1]))

    def forwardPropagation(self,X):
        layers = len(self.weights)
        H =[0 for i in range(layers-1)]
        A =[0 for i in range(layers)]
        for i in range(layers-1):
            if i==0:
                A[i] = np.add(np.dot(self.weights[i],X),self.biases[i])
                H[i] = self.activation(A[i])
            else:
                A[i] = np.add(np.dot(self.weights[i],H[i-1]),self.biases[i])
                H[i] = self.activation(A[i])
        A[layers-1] = np.add(np.dot(self.weights[layers-1],H[layers-2]),self.biases[layers-1])
        #print(A[layers-1])
        pred = softmax(A[layers-1])
        return pred,H,A

    def backPropagation(self, pred, weights, H, A, x_train, y_train):
        dW = []
        dB = []
        # x_train has the shape of 784*batch size
        #output layer gradient
        #if self.lossFunction =='CROSS':
        gradients ={}
        l = len(self.layersSize)-2
        # print(l)
        if self.lossFunction =='cross':
            gradients['a'+str(l)] = -(y_train - pred)
        elif self.lossFunction=='mse':
            gradients['a'+str(l)] = np.multiply(2*(pred-y_train),np.multiply(pred,(1-pred))) 
        #print(weights[0])
        # print(l)
        for i in range(l,0,-1):
            dw = np.dot(gradients['a'+str(i)],H[i-1].T)
            db = np.sum(gradients['a'+str(i)],axis=1).reshape(-1,1)
            dW.append(dw)
            dB.append(db)
            # print("iteration : {i}")
            # print(self.weights[i].shape)
            # print(gradients['a'+str(i)].shape)
            
            dh = np.matmul(weights[i].T,gradients['a'+str(i)])
            gradients['a'+str(i-1)] = np.multiply(dh,self.derivation_activation(A[i-1]))
        dW.append(np.dot(gradients['a'+str(0)],x_train.T))
        dB.append(np.sum(gradients['a'+str(0)],axis =1).reshape(-1,1))
        dW.reverse()
        dB.reverse()

        for i in range(self.layers-1):
            dW[i] = dW[i] + self.weights[i]*self.weightDecay
            #dB[i] = dB[i] + self.biases[i]*self.weightDecay
        return dW,dB
    
    def sgd(self,weight_decay=0):
        iterations = self.epochs
        layers = self.layers

        totalData = self.X.shape[-1]
        for i in tqdm(range(iterations)):
            j =0
            dW=[]
            dB=[]
            while(j<totalData):
                pred,H,A  = self.forwardPropagation(self.X[:,j:j+self.batchSize])
                dW,dB = self.backPropagation(pred, self.weights, H, A, self.X[:,j:j+self.batchSize],self.Y[:,j:j+self.batchSize])
                # 10,batch size = pred.shape
                #print(pred.shape)
                j+=self.batchSize

                for k in range(layers-1):
                    self.weights[k] = self.weights[k] - self.lr*dW[k]
                    self.biases[k] = self.biases[k] - self.lr*dB[k]
            train_acc, train_loss = self.accuracyLoss(self.X,self.y_train)
            val_acc, val_loss = self.accuracyLoss(self.X_val,self.y_val)
            #print(type(train_acc),type(train_loss),type(val_acc),type(val_loss))
            print("Train Accuracy - %.5f, Val Accuracy - %.5f, Train Loss - %.5f, Val Loss - %.5f,  EPOCH ==> %d"%(train_acc,val_acc,train_loss,val_loss,i+1))
            wb.log({"Train_Accuracy":train_acc,"Val_Accuracy":val_acc,"Train_Loss":train_loss,"Val_Loss":val_loss,"epoch":i})
            if((train_acc>0.09000 and train_acc<=0.10000) or (val_acc>0.09000 and val_acc<=0.01000) ):
                break



    def mgd(self,beta):
        #print(self.derivation_activation)
        iterations = self.epochs
        layers = self.layers
        uW = [0 for i in range(layers-1)]
        uB = [0 for i in range(layers-1)]
        totalsamples = self.X.shape[-1]
        for i in tqdm(range(iterations)):
            j =0
            dW =[]
            dB =[]
            while(j<totalsamples):
                pred, H, A = self.forwardPropagation(self.X[:,j:j+self.batchSize])
                dW, dB = self.backPropagation(pred, self.weights, H, A, self.X[:,j:j+self.batchSize],self.Y[:,j:j+self.batchSize])
                j+=self.batchSize

                for k in range(layers-1):
                    uW[k] = uW[k]*beta + dW[k]
                    uB[k] = uB[k]*beta + dB[k]
                
                for k in range(layers-1):
                    self.weights[k] -= self.lr*uW[k]
                    self.biases[k] -= self.lr*uB[k]

            train_acc, train_loss = self.accuracyLoss(self.X,self.y_train)
            val_acc, val_loss = self.accuracyLoss(self.X_val,self.y_val)
            #print(type(train_acc),type(train_loss),type(val_acc),type(val_loss))
            print("Train Accuracy - %.5f, Val Accuracy - %.5f, Train Loss - %.5f, Val Loss - %.5f,  EPOCH ==> %d"%(train_acc,val_acc,train_loss,val_loss,i+1))
            wb.log({"Train_Accuracy":train_acc,"Val_Accuracy":val_acc,"Train_Loss":train_loss,"Val_Loss":val_loss,"epoch":i})
            if((train_acc>0.09000 and train_acc<=0.10000) or (val_acc>0.09000 and val_acc<=0.01000) ):
                break    



    def nag(self,beta):
        iterations = self.epochs
        layers = self.layers
        print(layers)
        vW = [0 for i in range(layers-1)]
        vB = [0 for i in range(layers-1)]
        pvW = [0 for i in range(layers-1)]
        pvB = [0 for i in range(layers-1)]
        v_W = [0 for i in range(layers-1)]
        v_B = [0 for i in range(layers-1)]
        totalpoints = self.X.shape[-1]
        for i in tqdm(range(iterations)):
            j =0
            dW =[]
            dB =[]
            for k in range(layers-1):
                v_W[k] = beta*pvW[k]
                v_B[k] = beta*pvB[k]
            while(j<totalpoints):
                nw=[]
                pred, H, A = self.forwardPropagation(self.X[:,j:j+self.batchSize])
                for p in range(layers-1):
                    nw.append(self.weights[p] - v_W[p])
                dW,dB = self.backPropagation(pred, nw, H, A, self.X[:,j:j+self.batchSize],self.Y[:,j:j+self.batchSize])

                for l in range(layers-1):
                    vW[l] = beta*pvW[l] + self.lr*dW[l]
                    self.weights[l] -= vW[l]
                    vB[l] = beta*pvB[l] + self.lr*dB[l]
                    self.biases[l] -= vB[l]
                pvW = vW
                pvB = vB
                j+=self.batchSize
            train_acc, train_loss = self.accuracyLoss(self.X,self.y_train)
            val_acc, val_loss = self.accuracyLoss(self.X_val,self.y_val)
            #print(type(train_acc),type(train_loss),type(val_acc),type(val_loss))
            print("Train Accuracy - %.5f, Val Accuracy - %.5f, Train Loss - %.5f, Val Loss - %.5f,  EPOCH ==> %d"%(train_acc,val_acc,train_loss,val_loss,i+1))
            wb.log({"Train_Accuracy":train_acc,"Val_Accuracy":val_acc,"Train_Loss":train_loss,"Val_Loss":val_loss,"epoch":i})
            if((train_acc>0.09000 and train_acc<=0.10000) or (val_acc>0.09000 and val_acc<=0.01000) ):
                break


    def rmsProp(self,beta):
        
        layers = self.layers                    
        print(layers)
        vW = [0 for i in range(layers-1)]
        vB = [0 for i in range(layers-1)]
        eps =1e-4
        totalsamples = self.X.shape[-1]
        for i in tqdm(range(self.epochs)):
            j =0
            while(j<totalsamples):
                pred, H, A = self.forwardPropagation(self.X[:,j:j+self.batchSize])
                dW, dB = self.backPropagation(pred, self.weights,H, A, self.X[:,j:j+self.batchSize],self.Y[:,j:j+self.batchSize])


                for k in range(layers-1):
                    vW[k] = beta*vW[k] + (1-beta)*(dW[k]**2)
                    vB[k] = beta*vB[k] + (1-beta)*(dB[k]**2)
                for k in range(layers-1):
                    self.weights[k] -= self.lr*(dW[k]/(np.sqrt(vW[k])+eps))
                    self.biases[k] -= self.lr*(dB[k]/(np.sqrt(vB[k])+eps))
                
                j+=self.batchSize
            
            train_acc, train_loss = self.accuracyLoss(self.X,self.y_train)
            val_acc, val_loss = self.accuracyLoss(self.X_val,self.y_val)
            #print(type(train_acc),type(train_loss),type(val_acc),type(val_loss))
            print("Train Accuracy - %.5f, Val Accuracy - %.5f, Train Loss - %.5f, Val Loss - %.5f,  EPOCH ==> %d"%(train_acc,val_acc,train_loss,val_loss,i+1))
            wb.log({"Train_Accuracy":train_acc,"Val_Accuracy":val_acc,"Train_Loss":train_loss,"Val_Loss":val_loss,"epoch":i})
            if((train_acc>0.09000 and train_acc<=0.10000) or (val_acc>0.09000 and val_acc<=0.01000) ):
                break            


    def adam(self,beta1,beta2):
        layers = self.layers
        m_w,m_b,v_w,v_b = [[0 for l in range(layers-1)] for k in range(4)]
        totalsamples = self.X.shape[-1]
        eps = 1e-10
        for i in range(self.epochs):
            j =0

            while(j<totalsamples):
                pred, H, A = self.forwardPropagation(self.X[:,j:j+self.batchSize])
                dW, dB = self.backPropagation(pred, self.weights, H, A, self.X[:,j:j+self.batchSize],self.Y[:,j:j+self.batchSize])

                j+=self.batchSize

                for p in range(layers-1):
                    m_w[p] = beta1*m_w[p] + (1-beta1)*dW[p]
                    m_b[p] = beta1*m_b[p] + (1-beta1)*dB[p]
                    v_w[p] = beta2*v_w[p] + (1-beta2)*(dW[p]**2)
                    v_b[p] = beta2*v_b[p] + (1-beta2)*(dB[p]**2)
                m_w_hat,m_b_hat,v_b_hat,v_w_hat = [[0 for c in range(layers-1)] for q in range(4)]
                for c in range(layers-1):
                    m_w_hat[c] = m_w[c]/(1-np.power(beta1,i+1))
                    m_b_hat[c] = m_b[c]/(1-np.power(beta1,i+1))
                    v_w_hat[c] = v_w[c]/(1-np.power(beta2,i+1))
                    v_b_hat[c] = v_b[c]/(1-np.power(beta2,i+1))

                for c in range(layers-1):
                    self.weights[c] -= self.lr*m_w_hat[c]/(np.sqrt(v_w_hat[c])+eps)
                    self.biases[c] -= self.lr*m_b_hat[c]/(np.sqrt(v_b_hat[c])+eps)
            
            train_acc, train_loss = self.accuracyLoss(self.X,self.y_train)
            val_acc, val_loss = self.accuracyLoss(self.X_val,self.y_val)
            #print(type(train_acc),type(train_loss),type(val_acc),type(val_loss))
            print("Train Accuracy - %.5f, Val Accuracy - %.5f, Train Loss - %.5f, Val Loss - %.5f,  EPOCH ==> %d"%(train_acc,val_acc,train_loss,val_loss,i+1))
            wb.log({"Train_Accuracy":train_acc,"Val_Accuracy":val_acc,"Train_Loss":train_loss,"Val_Loss":val_loss,"epoch":i})
            if((train_acc>0.09000 and train_acc<=0.10000) or (val_acc>0.09000 and val_acc<=0.01000) ):
                break


    def nadam(self,beta1,beta2):
        layers = self.layers
        m_w,m_b,v_w,v_b = [[0 for i in range(layers-1)] for k in range(4)]
        eps = 1e-10
        totalsamples = self.X.shape[-1]
        for i in range(self.epochs):
            j = 0
            while(j<totalsamples):
                pred, H, A = self.forwardPropagation(self.X[:,j:j+self.batchSize])
                dW, dB = self.backPropagation(pred, self.weights, H, A, self.X[:,j:j+self.batchSize],self.Y[:,j:j+self.batchSize])
                j+=self.batchSize

                for p in range(layers-1):            
                    m_w[p] = beta1*m_w[p]+(1-beta1)*dW[p]
                    m_b[p] = beta1*m_b[p]+(1-beta1)*dB[p]
                    v_w[p] = beta2*v_w[p]+(1-beta2)*dW[p]**2
                    v_b[p] = beta2*v_b[p]+(1-beta2)*dB[p]**2

                m_w_hat,m_b_hat,v_b_hat,v_w_hat = [[0 for c in range(layers-1)] for q in range(4)]
                for c in range(layers-1):
                    m_w_hat[c] = m_w[c]/(1-np.power(beta1,i+1))
                    m_b_hat[c] = m_b[c]/(1-np.power(beta1,i+1))
                    v_w_hat[c] = v_w[c]/(1-np.power(beta2,i+1))
                    v_b_hat[c] = v_b[c]/(1-np.power(beta2,i+1))
                    
                for c in range(layers-1):
                    self.weights[c] -= (self.lr/np.sqrt(v_w_hat[c]+eps))*(beta1*m_w_hat[c]+(1-beta1)*dW[c]/(1-beta1**(i+1)))
                    self.biases[c] -= (self.lr/np.sqrt(v_b_hat[c]+eps))*(beta1*m_b_hat[c]+(1- beta1)*dB[c]/(1-beta1**(i+1)))
                
            
            train_acc, train_loss = self.accuracyLoss(self.X,self.y_train)
            val_acc, val_loss = self.accuracyLoss(self.X_val,self.y_val)
            #print(type(train_acc),type(train_loss),type(val_acc),type(val_loss))
            print("Train Accuracy - %.5f, Val Accuracy - %.5f, Train Loss - %.5f, Val Loss - %.5f,  EPOCH ==> %d"%(train_acc,val_acc,train_loss,val_loss,i+1))
            wb.log({"Train_Accuracy":train_acc,"Val_Accuracy":val_acc,"Train_Loss":train_loss,"Val_Loss":val_loss,"epoch":i})
            if((train_acc>0.09000 and train_acc<=0.10000) or (val_acc>0.09000 and val_acc<=0.10000) ):
                break


In [4]:
def main():
    wb.init(project="CS6910 - Assignment 1")
    config=wb.config
    name = "opt_"+str(config.optimizers)+"_init_"+str(config.initialization)+"_layers_"+str(config.number_of_hidden_layers)+"_sizeOfHL_"+str(config.neurons_in_hidden_layers)+"_lr_"+str(config.learning_rate)+"_activations_"+str(config.activations)+"_batch_"+str(config.batch_size)
    wb.run.name = name
    (train_x,train_y),(test_x,test_y) = fashion_mnist.load_data()
    classes =['Ankle boot','T-shirt/top','Dress','Pullover','sneaker','Sandal','Trouser','Shirt','Coat','Bag']
    split = 0.1
    total_data = train_x.shape[0]
    indices = np.arange(total_data)
    np.random.shuffle(indices)
    train_x = train_x[indices]
    train_y = train_y[indices]
    data_train = int((1-split)*total_data)
    x_train = train_x[:data_train]
    y_train = train_y[:data_train]
    x_val = train_x[data_train:]
    y_val = train_y[data_train:]  
    # layers, sizeHL, x_train, y_train, x_val, y_val, x_test, y_test, optimizer, batchSize, lr, iterations, activation,initializer, loss, weightDecay
    ob=FeedForwardNN(layers=config.number_of_hidden_layers,sizeHL=config.neurons_in_hidden_layers,x_train=x_train,y_train=y_train,x_val=x_val,y_val=y_val,x_test=test_x,y_test=test_y,batchSize=config.batch_size,lr=config.learning_rate,iterations=config.epochs,activation=config.activations,initializer=config.initialization,loss='cross',weight_decay=config.weight_decay)
    ob.train(optimizer=config.optimizers)



In [6]:
sweep_configuration = {
    'method': 'bayes',
    'name': 'run_34',
    'metric': {
        'goal': 'maximize',
        'name': 'validation_accuracy'
        },
    'parameters': {

        'initialization': {'values': ['xavier','random']},
        'number_of_hidden_layers' : {'values' : [2,3,4,5]},
        'neurons_in_hidden_layers' : {'values' : [32,64,128]},

        'learning_rate': {'values':[1e-1,5e-1,1e-2,1e-3,5e-3,1e-4]},
        'beta_value' : {'values' : [0.9,0.999]},
        'optimizers' : {'values' : ['sgd','mgd','rmsprop','adam','nadam','nag']},

        'batch_size': {'values': [32,64,128]},
        'epochs': {'values': [10]},
        'loss_type' : {'values' : ['cross']},
        'activations' : {'values' : ['sigmoid','relu','tanh']},
        'weight_decay' : {'values' : [0, 0.0005,0.5]}
       }
    }

sweep_id = wb.sweep(sweep=sweep_configuration,project='CS6910 - Assignment 1')

wb.agent(sweep_id , function = main , count = 250)
wb.finish()

Create sweep with ID: rkfp8ata
Sweep URL: https://wandb.ai/deeplearning-assignment/CS6910%20-%20Assignment%201/sweeps/rkfp8ata


wandb: Agent Starting Run: 3q647tnq with config:
wandb: 	activations: tanh
wandb: 	batch_size: 64
wandb: 	beta_value: 0.9
wandb: 	epochs: 10
wandb: 	initialization: xavier
wandb: 	learning_rate: 0.001
wandb: 	loss_type: cross
wandb: 	neurons_in_hidden_layers: 32
wandb: 	number_of_hidden_layers: 4
wandb: 	optimizers: rmsprop
wandb: 	weight_decay: 0.5
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


6


 10%|████▎                                      | 1/10 [00:01<00:15,  1.77s/it]

Train Accuracy - 0.55928, Val Accuracy - 0.55467, Train Loss - 10.97486, Val Loss - 9.90658,  EPOCH ==> 1


...

Train_Accuracy,▁
Train_Loss,▁
Val_Accuracy,▁
Val_Loss,▁
epoch,▁
Train_Accuracy,0.1002
Train_Loss,11.34303
Val_Accuracy,0.09817
Val_Loss,10.5321
epoch,0


wandb: Agent Starting Run: hqloqps2 with config:
wandb: 	activations: relu
wandb: 	batch_size: 32
wandb: 	beta_value: 0.999
wandb: 	epochs: 10
wandb: 	initialization: xavier
wandb: 	learning_rate: 0.1
wandb: 	loss_type: cross
wandb: 	neurons_in_hidden_layers: 32
wandb: 	number_of_hidden_layers: 5
wandb: 	optimizers: mgd
wandb: 	weight_decay: 0.0005
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


  0%|                                                   | 0/10 [00:02<?, ?it/s]

Train Accuracy - 0.09941, Val Accuracy - 0.10533, Train Loss - nan, Val Loss - nan,  EPOCH ==> 1


wandb: Network error (ConnectionError), entering retry loop.


In [6]:


layers = 4
sizeHL =[512,128,64,32]
optimizer = 'sgd'
batchSize = 200
weight_decay = 0
lr = 0.0001
iterations = 100
activation = 'tanh'
initializer = 'xavier'
loss= 'cross'
momentum = 0.4
beta1 = 0.5
beta2 = 0.5


print(x_train.shape,y_train.shape)
print(x_val.shape,y_val.shape)


(54000, 28, 28) (54000,)
(6000, 28, 28) (6000,)


In [8]:

ob = FeedForwardNN(layers,sizeHL,x_train,y_train,x_val,y_val,test_x,test_y,optimizer,batchSize,lr,iterations,activation,initializer,loss, weight_decay)
ob.mgd(momentum)

  1%|▍                                         | 1/100 [00:09<14:54,  9.03s/it]

Train Accuracy - 0.53622, Val Accuracy - 0.53667, Train Loss - 10.96498, Val Loss - 9.79393,  EPOCH ==> 1


  2%|▊                                         | 2/100 [00:21<18:30, 11.33s/it]

Train Accuracy - 0.53639, Val Accuracy - 0.53100, Train Loss - 10.96949, Val Loss - 9.83463,  EPOCH ==> 2


  2%|▊                                         | 2/100 [00:27<22:43, 13.91s/it]


KeyboardInterrupt: 